In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import *
conf = SparkConf().setAppName("ALS")
conf = (conf.setMaster('local[*]')\
        .set('spark.executor.memory','4G')\
       .set('spark.driver.memory','6G')\
        .set('spark.driver.maxResultSize','6G'))
sc = SparkContext(conf=conf)

In [2]:
def parse_artist(x):
    try:
        return int(x)
    except:
        return []

In [3]:
rawArtistData = sc.textFile("artist_data.txt")

In [9]:
artistByID = rawArtistData.map(lambda x: x.split('\t'))\
                        .filter(lambda x: len(x) == 2)\
                          .map(lambda x: [parse_artist(x[0]), x[1]])\
                            .filter(lambda x: len(x) == 2)

In [10]:
def parse_alias(x,y):
    try:
        return [int(x), int(y)]
    except:
        return []

In [11]:
rawArtistAlias = sc.textFile("artist_alias.txt")

In [13]:
artistAlias = rawArtistAlias.map(lambda x: x.split('\t'))\
                            .map(lambda x: parse_alias(x[0], x[1]))\
                            .filter(lambda x : len(x) == 2)\
                                    .collectAsMap()

In [14]:
def convert_to_goodId(x):
    if x[1] in artistAlias.keys():
        x[1] = artistAlias.get(x[1])
    return [x[0], x[1], x[2]]

In [15]:
rawUserArtistData = sc.textFile("user_artist_data.txt")

In [16]:
UserArtistData = rawUserArtistData.map(lambda x: x.split(' '))\
                                  .map(lambda x: [int(x[0]), int(x[1]), int(x[2])])\
                                    .map(convert_to_goodId)\
                                .cache()

In [17]:
model = ALS.trainImplicit(UserArtistData, 10, 5, 0.01)

In [18]:
recommendation = model.recommendProducts (2093760, 10)

In [19]:
recommendation

[Rating(user=2093760, product=1007614, rating=0.0346008395534953),
 Rating(user=2093760, product=4605, rating=0.032679092918638145),
 Rating(user=2093760, product=2814, rating=0.031085591831998136),
 Rating(user=2093760, product=1037970, rating=0.03082570465712907),
 Rating(user=2093760, product=1001819, rating=0.030461079105161826),
 Rating(user=2093760, product=829, rating=0.03030720472060452),
 Rating(user=2093760, product=1811, rating=0.02956754457010531),
 Rating(user=2093760, product=250, rating=0.029544653947789552),
 Rating(user=2093760, product=1003249, rating=0.027312072660190652),
 Rating(user=2093760, product=868, rating=0.025811307660946203)]

In [23]:
artistByID.contains(1007614)

AttributeError: 'PipelinedRDD' object has no attribute 'contains'

In [16]:
# artistMap=artistByID.collectAsMap()
# for i in recommendation:
#     print(artistByID.get(i.product), i.rating)

TypeError: unhashable type: 'list'